<a href="https://colab.research.google.com/github/zqian/google_colab_scripts/blob/main/Rubric_Item_Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Based on this [Canvas Idea Conversation](https://community.canvaslms.com/t5/Idea-Conversations/Export-aggregate-rubric-scores-to-identify-strengths-and/idi-p/370265/page/3?emcs_t=S2h8ZW1haWx8ZGlnZXN0X25vdGlmaWNhdGlvbnxLWU9HSEw3TFIzVVpFOHwtMXxPVEhFUlN8aEs#comments), here is a PoC script to generate a csv file that shows the rubric line item scores for all student submissions.

**Install Python Libraries**

In [ ]:
# install all libraries
!pip install canvasapi
!pip install future-fstrings
!pip install pandas

**Step 1: get Canvas credentials**

*   Log into Canvas instance
*   Go to "Account"-> "Settings" -> "+ New Access Token". You can name it as "Rubric Ttem Report", and leave the "Expires" field empty for no expiration. Click on "Generate Token"
*   Copy the token into next section

In [ ]:
from canvasapi import Canvas
from canvasapi.assignment import (
    Assignment,
    AssignmentExtension,
    AssignmentGroup,
    AssignmentOverride,
)

from canvasapi.rubric import Rubric, RubricAssociation
from canvasapi.submission import GroupedSubmission, Submission
import pandas as pd

from google.colab import userdata
CANVAS_URL = userdata.get('CANVAS_URL')
CANVAS_API_KEY = userdata.get('CANVAS_API_KEY')
CANVAS_COURSE_ID = userdata.get('CANVAS_COURSE_ID')
CANVAS_ASSIGNMENT_ID = userdata.get('CANVAS_ASSIGNMENT_ID')
CANVAS_QUIZ_ID = userdata.get('CANVAS_QUIZ_ID')

**Step 2: Retrieve assignment and its rubric settings**

In [ ]:
# for quiz
# init CanvasAPI library
# get course, quiz with rubric settings
canvas = Canvas(CANVAS_URL, CANVAS_API_KEY)
course = canvas.get_course(CANVAS_COURSE_ID)

# NOTE: if you start with quiz, make sure to get its associated assignment first
quiz = course.get_quiz(CANVAS_QUIZ_ID, include=['overrides'])
print(quiz.title)
print(quiz.quiz_type)
assignment_id = quiz.assignment_id
print(assignment_id)
assignment = course.get_assignment(assignment_id)
assignment_name = assignment.name
print(assignment_name)

In [ ]:
 # Otherwise, get the assignment based on assignment id

assignment = course.get_assignment(CANVAS_ASSIGNMENT_ID, include=['overrides'])
assignment_name = assignment.name

In [ ]:

# get assignment rubric settings object; retrieve the rubric id
assignment_rubric_settings = assignment.rubric_settings
assignment_rubric_id = assignment_rubric_settings['id']
assignment_rubric_title = assignment_rubric_settings['title']
assignment_rubric_points_possible = assignment_rubric_settings['points_possible']

# get rubric settings
rubric_settings = assignment.rubric
for setting in rubric_settings:
  print(f"{setting['id']} {setting['description']}")

**Step 3: iterating through assignment submissions. Output csv report file for each rubric criteria**

In [ ]:
from google.colab import files
import os

submissions = assignment.get_submissions(include=['rubric_assessment'])

for setting in rubric_settings:
  print(f"{setting['id']} {setting['description']}")
  rubric_item_id = setting['id']
  rubric_item_description = setting['description']

  # the output csv file columns
  # you can modify the column to include other attributes, e.g. student name, points_possible, etc
  df = pd.DataFrame(columns = ["user_id", "points", "comments"])
  for submission in submissions:
    try:
      if submission.rubric_assessment:
        # Access rubric assessment data within a try-except block
        try:
          points = submission.rubric_assessment[rubric_item_id]['points']
        except KeyError:
          points = float('nan')  # Assign NaN for missing points
        try:
          comments = submission.rubric_assessment[rubric_item_id]['comments']
        except KeyError:
          comments = ''  # Assign empty string for missing comments

        # Now you can create the DataFrame row using the obtained values
        row = {"user_id": submission.user_id,
              "points": points,
              "comments": comments}

        new_row = pd.DataFrame(row, index=[0])  # Provide an index
        df = pd.concat([df, new_row], ignore_index=True)
    except Exception as e:
      print(f"no rubric assessment for submission {submission}: {e}")

  # the output file is named after "{assignment_name}_{rubric_item_description}_report.csv"
  # you can change the file name format acoordingly
  download_file_name = f"{assignment_name}_{rubric_item_description}_report.csv"
  # Sanitize the filename by replacing invalid characters
  download_file_name = "".join(c for c in download_file_name if c.isalnum() or c in ("_", ".", "-")).rstrip()

  # Create the directory if it doesn't exist
  directory = os.path.dirname(download_file_name)
  if directory and not os.path.exists(directory):
    os.makedirs(directory)
  df.to_csv(download_file_name, index=False)
  files.download(download_file_name)